In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = (
    pl.DataFrame(
        {
            "f": np.sin(list(range(size))),
            "time_idx": range(size),
            "dummy": ["a"] * (size // 2) + ["b"] * (size // 2),
            "actual": np.round(np.random.random(size=size)).astype(np.int32),
            "predicted": np.random.random(size=size),
            "dummy_groups": ["a"] * (size // 2) + ["b"] * (size // 2),
        }
    )
    .with_columns(
        pds.random(0.0, 1.0).alias("x1"),
        pds.random(0.0, 1.0).alias("x2"),
        pds.random(0.0, 1.0).alias("x3"),
        pds.random(0.0, 1.0).alias("a"),
        pds.random(0.0, 1.0).alias("b"),
    )
    .with_columns(
        y=pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
        y2=pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001,
    )
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.865246,"""a""",0.531304,0.342598,0.555673,0.141504,0.374844,-0.651013,0.167762
0.841471,1,"""a""",1,0.426308,"""a""",0.66626,0.767732,0.319284,0.989981,0.257848,-0.148656,0.400214
0.909297,2,"""a""",1,0.023306,"""a""",0.52806,0.2415,0.574577,0.492929,0.427643,-0.710107,0.119897
0.14112,3,"""a""",0,0.890483,"""a""",0.92675,0.35313,0.332102,0.95379,0.06414,-0.253124,0.246243
-0.756802,4,"""a""",1,0.804295,"""a""",0.124174,0.028771,0.580792,0.9729,0.466718,-0.843863,-0.02891


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(pds.jaccard_col("x1", pl.col("x2")))

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant
# compact representation of fft output.
df.select(pds.rfft("f")).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(pds.rfft("f", return_full=True)).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve(
        "f", [-1, 0, 0, 0, 1], mode="full", method="fft"
    ),  # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode="full", method="direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode="full", method="direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.141504,-0.374844
-0.841471,-0.989981,-0.257848
-0.909297,-0.492929,-0.427643
-0.14112,-0.95379,-0.06414
0.756802,-0.831396,-0.091874


In [7]:
# Linear Regression
df.select(pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False))

coeffs
list[f64]
"[-0.49286, -0.338774]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=[pl.col("y"), pl.col("y2")], add_bias=False)
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.49286, -0.338774]","[0.087182, 0.407455]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=[pl.col("y"), pl.col("y2")], add_bias=False)
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.49286, -0.338774]","[0.087182, 0.407456]"


In [ ]:
pds.Config.LIN_REG_EXPR_F64 = True  # pds.Config or pds.config will both work

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries,
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)",
        "exp(x2)",
        "sin(x3)",
        target="y",
        add_bias=True,
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.216885,0.001695,127.979002,0.0,0.213563,0.220207,0.99432,0.994318
"""exp(x2)""",0.174897,0.000678,257.890455,0.0,0.173567,0.176226,0.99432,0.994318
"""sin(x3)""",-1.744464,0.00135,-1292.580898,0.0,-1.747109,-1.741818,0.99432,0.994318
"""__bias__""",-0.107621,0.001511,-71.206374,0.0,-0.110583,-0.104658,0.99432,0.994318


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries,
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)",
        "exp(x2)",
        "sin(x3)",
        target="y",
        add_bias=True,
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.216881,0.001695,127.976707,0.0,0.213559,0.220203,0.99432,0.994318
"""exp(x2)""",0.174897,0.000678,257.890411,0.0,0.173567,0.176226,0.99432,0.994318
"""sin(x3)""",-1.744467,0.00135,-1292.583496,0.0,-1.747113,-1.741822,0.99432,0.994318
"""__bias__""",-0.10762,0.001511,-71.205765,0.0,-0.110582,-0.104657,0.99432,0.994318


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False).over(
        pl.col("dummy")
    ),
)

dummy,coeffs
str,list[f64]
"""a""","[-0.492705, -0.347587]"
"""a""","[-0.492705, -0.347587]"
"""a""","[-0.492705, -0.347587]"
"""a""","[-0.492705, -0.347587]"
"""a""","[-0.492705, -0.347587]"
…,…
"""b""","[-0.49306, -0.329949]"
"""b""","[-0.49306, -0.329949]"
"""b""","[-0.49306, -0.329949]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg("x1", pl.col("x2"), target="y", add_bias=False, return_pred=True).alias(
        "prediction"
    ),
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.531304,0.342598,-0.651013,-0.377922,-0.273091
0.66626,0.767732,-0.148656,-0.58846,0.439805
0.52806,0.2415,-0.710107,-0.342073,-0.368034
0.92675,0.35313,-0.253124,-0.57639,0.323266
0.124174,0.028771,-0.843863,-0.070947,-0.772915


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False)
)

dummy,coeffs
str,list[f64]
"""a""","[-0.492705, -0.347587]"
"""b""","[-0.49306, -0.329949]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), l1_reg=0.1, add_bias=False)
)

dummy,coeffs
str,list[f64]
"""b""","[-0.322837, -0.156869]"
"""a""","[-0.316503, -0.177763]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual=pl.col("y"),
        pred=pds.lin_reg(
            pl.col("x1"),
            pl.col("x2"),
            target=pl.col("y"),
            l1_reg=0.1,
            return_pred=True,
            add_bias=False,
        ).struct.field("pred"),
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.536532
"""b""",-0.514658


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg("x1", "x2", target="y", window_size=5, null_policy="zero").alias("result"),
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.651013,0.531304,0.342598,null,null
-0.148656,0.66626,0.767732,null,null
-0.710107,0.52806,0.2415,null,null
-0.253124,0.92675,0.35313,null,null
-0.843863,0.124174,0.028771,"[-0.90984, 0.477997]",-0.099226
…,…,…,…,…
-0.147426,0.931717,0.147461,"[-0.332345, -0.48363]",-0.380968
0.274645,0.966595,0.932579,"[-0.26154, 0.0467]",-0.209252
-0.591685,0.472488,0.26536,"[-0.647542, 0.698849]",-0.12051


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(pds.query_cond_entropy("y", "x1"))

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(pds.singular_values("a", "b", "x1"))

a
list[f64]
"[29.123781, 28.848938, 28.689624]"


In [22]:
# Singular values + The principal components
df.select(pds.pca("a", "b")).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.047303,"[0.9709, -0.239487]"
28.82667,"[0.239487, 0.9709]"


In [23]:
# PC1
df.select(pds.principal_components("a", "b", k=1).alias("principal_components")).unnest(
    "principal_components"
).head()

pc1
f64
-0.316118
0.535687
0.012437
0.546941
0.469082


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo"),
).unnest("combo")

dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.329661,0.997781,0.5,0.501617,0.500807,0.500811,0.505487
"""a""",0.338734,1.021778,0.486155,0.501019,0.493475,0.478556,0.490244


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame(
    {"sen": ["Hello, world! I'm going to church."] * size, "word": ["words", "word"] * (size // 2)}
)
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
df2.select(pds.str_leven("word", pl.lit("world"))).head()

word
u32
2
1
2
1
2


In [27]:
# Damerau-Levenshtein
df2.select(pds.str_d_leven("word", pl.lit("world"))).head()

word
u32
2
1
2
1
2


In [28]:
df2.select(  # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim=True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [29]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1)  # <= 1.
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [30]:
df = pl.DataFrame(
    {
        "word": ["apple", "banana", "pineapple", "asasasas", "sasasass"],
        "other_data": [1, 2, 3, 4, 5],
    }
)
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [31]:
df.select(
    # Nearest string
    pds.str_nearest("word", word="banana")
)

word
str
"""banana"""


In [32]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab=gibberish,
        threshold=0.5,
        metric="lv",  # Levenshtein similarity. Other options: dleven, osa, jw
        strategy="any",  # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [33]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias(
        "Optimal String Alignment - apples"
    ),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)

asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [34]:
import numpy as np

df = pl.DataFrame({"a": [None, None] + list(np.random.normal(size=998))})
df.head()

a
f64
null
null
0.297652
-1.085844
1.623945


In [35]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean=0.5, std=1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_normal(mean=0.5, std=1.0).alias("random_normal"))
    .alias("random_normal_that_respects_null_of_a"),
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.581644,null
null,0.731812,null
0.297652,-0.133626,-0.916043
-1.085844,-0.685907,1.876444
1.623945,0.149713,0.4434


In [36]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size=1, max_size=5).alias("random_str"),
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_str(min_size=1, max_size=5))
    .alias("random_str_that_respects_null_of_a"),
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Jld""",null
null,"""d0Xpa""",null
0.297652,"""DV""","""f8"""
-1.085844,"""Nb""","""p"""
1.623945,"""H80o""","""y"""


In [37]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_str(min_size=5, max_size=5))
    .alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.297652,"""8InwE"""
-1.085844,"""ByGh8"""
1.623945,"""s8v53"""


In [38]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1"),
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random(lower=0.0, upper=pl.col("a").max()).alias("test2")),
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,0.079325,null,0.078981
null,-0.735392,null,-0.735726
0.297652,-0.534527,2.460246,-0.534622
-1.085844,0.341939,0.794185,0.341735
1.623945,-1.252153,0.312352,-1.252385


In [39]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.377581,-0.291951,5
null,0.158476,-2.229577,1
0.297652,0.375157,0.224972,1
-1.085844,0.971395,0.16292,3
1.623945,0.627268,1.558509,7


In [40]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test"),
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics"),
    pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue"),
    pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics"),
    pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue"),
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-1.727544,0.084277,2.140865,0.34286


In [41]:
size = 5_000
df = pl.DataFrame(
    {
        "market_id": range(size),
    }
).with_columns(
    pl.col("market_id").mod(3),
    var1=pds.random(),
    var2=pds.random(),
    category_1=pds.random_int(0, 5),
    category_2=pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.225594,0.429516,2,0
1,0.328704,0.081412,0,8
2,0.877838,0.517795,1,4
0,0.096504,0.143272,1,7
1,0.910098,0.856956,1,4


In [42]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group="category_1").alias("f-test"),
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.286603,0.774422}","{38.761962,0.346159}","{1.61827,0.166679}"


In [43]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group="category_1").alias("f-test"),
    )
)

shape: (3, 4)
┌───────────┬─────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test              ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                 ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]           ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪═════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-2.179,0.029402}   ┆ {26.677158,0.870887} ┆ {1.656147,0.157663} │
│ 1         ┆ {0.715089,0.474604} ┆ {38.900734,0.340448} ┆ {0.493891,0.740251} │
│ 2         ┆ {0.962678,0.335779} ┆ {50.352682,0.056579} ┆ {0.934368,0.442981} │
└───────────┴─────────────────────┴──────────────────────┴─────────────────────┘


In [44]:
# Benford's law
df.select(first_digit_cnt=pds.query_first_digit_cnt(pl.col("var1")).explode()).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution=pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
567,0.1134
538,0.1076
561,0.1122
576,0.1152
589,0.1178
574,0.1148
565,0.113
513,0.1026
517,0.1034


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [45]:
import polars_ds as pds

size = 2000
df = pl.DataFrame(
    {
        "id": range(size),
    }
).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32),
)

In [46]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"),
        "var2",
        "var3",  # Columns used as the coordinates in n-d space, str | pl.Expr
        r=0.1,  # radius
        dist="inf",  # L Infinity distance
        parallel=True,
    ).alias("nb_l_inf_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.579134,0.329014,0.84423,0.056414,5.167237,18
1,0.136451,0.115837,0.743302,0.684414,6.663641,16
2,0.858462,0.671112,0.379695,0.323486,8.680795,20
3,0.362156,0.541902,0.985665,0.265312,0.041498,15
4,0.411226,0.753546,0.811137,0.201864,6.942092,14


In [47]:
df.with_columns(
    pds.query_nb_cnt(
        "var1",
        "var2",
        "var3",  # Columns used as the coordinates in n-d space, str | pl.Expr
        r=pl.col("r"),  # radius be an expression too
        dist="l1",  # L 1 distance
        parallel=True,
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.579134,0.329014,0.84423,0.056414,5.167237,1
1,0.136451,0.115837,0.743302,0.684414,6.663641,430
2,0.858462,0.671112,0.379695,0.323486,8.680795,99
3,0.362156,0.541902,0.985665,0.265312,0.041498,33
4,0.411226,0.753546,0.811137,0.201864,6.942092,23


In [48]:
# Get ids of the k nearest neighbors.
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        index="id",  # pl.col("id"), str | pl.Expr
        k=3,
        dist="l2",  # squared l2
        parallel=True,
    ).alias("best friends")
).head()

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.579134,0.329014,0.84423,0.056414,5.167237,"[0, 1881, … 959]"
1,0.136451,0.115837,0.743302,0.684414,6.663641,"[1, 1730, … 172]"
2,0.858462,0.671112,0.379695,0.323486,8.680795,"[2, 1827, … 689]"
3,0.362156,0.541902,0.985665,0.265312,0.041498,"[3, 1479, … 965]"
4,0.411226,0.753546,0.811137,0.201864,6.942092,"[4, 1624, … 696]"


In [49]:
# Get all neighbors within radius r, call them best friends
print(
    df.select(
        pl.col("id"),
        pds.query_radius_ptwise(
            pl.col("var1"),
            pl.col("var2"),
            pl.col("var3"),  # Columns used as the coordinates in 3d space
            index=pl.col("id"),
            r=0.1,
            dist="l2",  # actually this is squared l2
            parallel=True,
        ).alias("best friends"),
    )
    .with_columns(  # -1 to remove the point itself
        (pl.col("best friends").list.len() - 1).alias("best friends count")
    )
    .head()
)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1881, … 1075] ┆ 11                 │
│ 1   ┆ [1, 1730, … 1737] ┆ 7                  │
│ 2   ┆ [2, 1827, … 1817] ┆ 9                  │
│ 3   ┆ [3, 1479, … 1545] ┆ 8                  │
│ 4   ┆ [4, 1624, … 1364] ┆ 6                  │
└─────┴───────────────────┴────────────────────┘


In [50]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        index=pl.col("id"),
        k=3,
        dist="l2",  # actually this is squared l2
        parallel=True,
        return_dist=True,
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.579134,0.329014,0.84423,0.056414,5.167237,"[0, 1881, … 959]","[0.0, 0.053066, … 0.065413]"
1,0.136451,0.115837,0.743302,0.684414,6.663641,"[1, 1730, … 172]","[0.0, 0.036583, … 0.077932]"
2,0.858462,0.671112,0.379695,0.323486,8.680795,"[2, 1827, … 689]","[0.0, 0.060563, … 0.072454]"
3,0.362156,0.541902,0.985665,0.265312,0.041498,"[3, 1479, … 965]","[0.0, 0.028195, … 0.066759]"
4,0.411226,0.753546,0.811137,0.201864,6.942092,"[4, 1624, … 696]","[0.0, 0.071199, … 0.080162]"


In [51]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        pt=[0.5, 0.5, 0.5],
        r=0.2,
        dist="l2",  # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.579134,0.329014,0.84423,0.056414,5.167237
2,0.858462,0.671112,0.379695,0.323486,8.680795
4,0.411226,0.753546,0.811137,0.201864,6.942092
5,0.099687,0.553936,0.375904,0.278505,1.227848
8,0.399726,0.501957,0.463459,0.761134,4.696983


In [52]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),  # Columns used as the coordinates in n-d space
        pt=[0.5, 0.5],
        r=10,  # in km
        dist="h",
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
14,0.528173,0.544334,0.69444,0.399148,7.458917
74,0.487442,0.576601,0.462924,0.048392,0.865228
76,0.45438,0.554271,0.542959,0.595356,9.977573
112,0.415284,0.471442,0.063427,0.728378,1.857436
116,0.583013,0.510799,0.021017,0.657546,6.362856


In [53]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),
        pt=[0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r=pl.col("rh"),
        dist="h",
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
14,0.528173,0.544334,0.69444,0.399148,7.458917
76,0.45438,0.554271,0.542959,0.595356,9.977573
123,0.50731,0.505499,0.873323,0.197635,3.340024
411,0.467528,0.53937,0.796454,0.18563,8.509126
533,0.526005,0.52802,0.625523,0.068372,7.109258


In [54]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"),
        pl.col("var2"),
        index=pl.col("id"),
        r=0.02,
        dist="l2",
    ).alias("friends"),
).with_columns(pl.col("friends").list.len().alias("count"))
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 12, … 340]",4
1,[1],1
2,"[2, 1263, … 1798]",5
3,"[3, 1786]",2
4,"[4, 595, 696]",3


# Compatibility

## Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures. 

## Using PDS with Narwhals

Limited

In [55]:
import pandas as pd
import numpy as np
import polars as pl
import polars_ds as pds
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_12078/181413171.py:5: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.624251,0,7,"""aG""","""Jb"""
1.0,0.330709,2,3,"""Gh""","""I"""
1.0,0.858817,1,1,"""8G""","""l"""
1.0,0.097989,0,1,"""j""","""Uv"""
0.0,0.545426,1,1,"""m""","""R"""


In [56]:
df_pd = df.to_pandas()

In [57]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

0    0.3
Name: , dtype: float64

In [58]:
# Polars Series
print(pds2.query_roc_auc(df["actual"], df["predicted"]))
# NumPy
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_numpy(), df["predicted"].to_numpy()))
pds2.return_numpy = False
# Pandas
print(pds2.query_roc_auc(df["actual"].to_pandas(), df["predicted"].to_pandas()))
# PyArrow
# Arrow series can be inputs, but the output cannot be converted correctly. Please let me know if you have a fix.
# The work around is to use NumPy for Arrow
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_arrow(), df["predicted"].to_arrow()))
# Other array-protocal compatible inputs
# print(pds2.query_roc_auc(df["actual"].to_jax(), df["predicted"].to_jax()))

pds2.return_numpy = False

shape: (1,)
Series: '' [f64]
[
	0.501098
]
[0.5010979]
0    0.501098
Name: , dtype: float64
[0.5010979]


In [59]:
# NumPy Arrays
pds2.psi(
    np.random.random(size=1000),
    np.random.random(size=1000),
    n_bins=5,
)

array([0.00602434])

In [60]:
df_pd = df.to_pandas()
df_pd["levenshtein_dist"] = pds2.str_leven(df_pd["s1"], df_pd["s2"])
df_pd.head()

,actual,predicted,0-2,0-9,s1,s2,levenshtein_dist
0,1.0,0.624251,0,7,aG,Jb,2
1,1.0,0.330709,2,3,Gh,I,2
2,1.0,0.858817,1,1,8G,l,2
3,1.0,0.097989,0,1,j,Uv,2
4,0.0,0.545426,1,1,m,R,1


In [61]:
# If you are using Narwhals, well, Narwhal expressions are not Polars expressions.
# Using the pds2 module, you can run pds functions in map_batches, but this is limited to 1 input column.

import narwhals as nw

df_nw = nw.from_native(df_pd)
df_nw.with_columns(
    nw_levenshtein_dist=nw.col("s1").map_batches(
        lambda s: pds2.str_leven(s.to_numpy(), pl.lit("k9"))
    )
).head()

┌─────────────────────────────────────────────────────────────────────────────┐
|                             Narwhals DataFrame                              |
|-----------------------------------------------------------------------------|
|   actual  predicted  0-2  0-9  s1  s2  levenshtein_dist  nw_levenshtein_dist|
|0     1.0   0.624251    0    7  aG  Jb                 2                    2|
|1     1.0   0.330709    2    3  Gh   I                 2                    2|
|2     1.0   0.858817    1    1  8G   l                 2                    2|
|3     1.0   0.097989    0    1   j  Uv                 2                    2|
|4     0.0   0.545426    1    1   m   R                 1                    2|
└─────────────────────────────────────────────────────────────────────────────┘